In [2]:
import numpy as np
import pandas as pd
import plotly.express as px
import subprocess
import os
import sys
sys.path.append('/usr/workspace/vanover1/approx-llvm/approx')
from approx_modules import approx

from sklearn.ensemble import ExtraTreesRegressor

/usr/tce/packages/python/python-3.7.2/lib/python3.7/site-packages/sklearn/ensemble/gradient_boosting.py:34: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  from ._gradient_boosting import predict_stages
/usr/tce/packages/python/python-3.7.2/lib/python3.7/site-packages/sklearn/ensemble/gradient_boosting.py:34: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecatio

In [3]:
N_KERNELS = 4
KERNEL_NAMES = ["s1", "t1", "t2", "t3"]

In [4]:
def run_arclength(endpoint):
    
    # compile and run arclength instrumented with HPAC directives
    # need to use /usr/tce/bin/git binary because the default git version grabbed by this subprocess is too early for the --show-superproject-working-tree flag
    subprocess.check_call("rm -f test.h5 && source $(/usr/tce/bin/git rev-parse --show-superproject-working-tree)/scripts/activate_hpac_env.sh && make arclength-hpac && ./arclength-hpac {}".format(endpoint), shell=True)
    
    # open database
    approxDataProfile = approx.approxApplication("./test.h5")
    
    # get output
    Y = approxDataProfile.getApplicationOutput()['arclength'][0]

    # get aggregated kernel outputs
    kernel_outputs = []
    for kernel_name in KERNEL_NAMES:
        kernel_outputs.append(approxDataProfile[kernel_name].Y().mean())
    
    return kernel_outputs, Y

In [5]:
# define neighborhood for arclength analysis
center = np.pi
eps = 0.1
n_sample = 2**6

# uniform sample of n_sample endpoints from the neighborhood
XX = np.linspace(center-eps, center+eps, n_sample)

In [6]:
# run arclength
kernel_outs = []
YY = []
for n, X in enumerate(XX):
    temp = run_arclength(X)
    kernel_outs.append(temp[0])
    YY.append(temp[1])
kernel_outs = np.array(kernel_outs)
YY = np.array(YY)

In [7]:
# analyze kernel output sensitivity
et = ExtraTreesRegressor(n_estimators=100,
                                    criterion="mse",
                                    # max_features=int(round(XX.shape[1] / 3)),
                                    max_depth=8,
                                    min_samples_split=2,
                                    min_samples_leaf=max(1, int(round(np.sqrt(kernel_outs.shape[0]) / np.sqrt(1000)))),
                                    min_weight_fraction_leaf=0,
                                    max_leaf_nodes=None,
                                    #bootstrap=True,
                                    #oob_score=True,
                                    random_state=1)
et.fit(kernel_outs, YY)
for k_no in range(N_KERNELS):
    print("{} : {}".format(KERNEL_NAMES[k_no], et.feature_importances_[k_no]))

s1 : 0.3336069360096248
t1 : 0.16583634525384497
t2 : 0.24247230423419439
t3 : 0.258084414502336


## Compare to:
```
=== BEGIN ADAPT REPORT ===
15000004 total independent/intermediate variables
1 dependent variables
Mixed-precision recommendation:
  Replace variable d2      max error introduced: 0.000000e+00  count: 6000000     totalerr: 0.000000e+00
  Replace variable d1      max error introduced: 0.000000e+00  count: 1           totalerr: 0.000000e+00
  DO NOT replace   t2      max error introduced: 8.623994e-08  count: 1000000     totalerr: 8.623994e-08
  DO NOT replace   t1      max error introduced: 8.623994e-08  count: 1000001     totalerr: 1.724799e-07
  DO NOT replace   h       max error introduced: 5.401810e-07  count: 1           totalerr: 7.126609e-07
  DO NOT replace   t3      max error introduced: 9.462050e-07  count: 6000000     totalerr: 1.658866e-06
  DO NOT replace   s1      max error introduced: 6.334703e-06  count: 1000001     totalerr: 7.993569e-06
=== END ADAPT REPORT ===
Arclength - SUCCESSFUL!
```